In [ ]:
!pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 10.7 MB/s eta 0:00:00


In [ ]:
!pip install rank-metrics

ERROR: Could not find a version that satisfies the requirement rank-metrics (from versions: none)
ERROR: No matching distribution found for rank-metrics


In [ ]:
import pandas as pd
import numpy as np


In [ ]:
from langchain_groq import ChatGroq
import os
import pandas as pd

# Set your API key
os.environ["GROQ_API_KEY"] = "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"

# Init LLM
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_tokens=100,
    max_retries=2,
)

# Load datasets
df = pd.read_csv('u.csv')
movies_sub = pd.read_csv('movies_sub.csv')

# Load previously saved predictions (if any)
csv_file = "predicted_rankings.csv"
predicted_rankings = {}

if os.path.exists(csv_file):
    existing_df = pd.read_csv(csv_file)
    predicted_rankings = dict(zip(existing_df["user_id"], existing_df["predicted_ranking"].apply(eval)))

# Buffer for batch saving
buffer = []

# Iterate over users
for idx, user_id in enumerate(df['user id'].unique()):
    if user_id in predicted_rankings:
        continue

    try:
        user_movies = df[df['user id'] == user_id]
        if len(user_movies) < 20:
            continue

        sorted_movies = user_movies.sort_values(by='rating', ascending=False)
        top_15 = sorted_movies.head(15)
        remaining_5 = sorted_movies.tail(5)

        top_15_summaries = movies_sub[movies_sub['Movie_ID'].isin(top_15['item id'])][['Movie_ID', 'Llama Summary Few Shot']]
        remaining_5_summaries = movies_sub[movies_sub['Movie_ID'].isin(remaining_5['item id'])][['Movie_ID', 'Llama Summary Few Shot']]

        if len(top_15_summaries) < 15 or len(remaining_5_summaries) < 5:
            continue

        prompt = "The following are 15 movies the user has liked the most, ranked from most liked to least liked. Learn the user's taste in terms of genres and themes based on their summaries:\n"
        for _, row in top_15_summaries.iterrows():
            prompt += f"Movie: {row['Movie_ID']}, Summary: {row['Llama Summary Few Shot']}\n"

        prompt += "\nNow, based on the user's genre and theme preferences learned above, rank the following 5 movies from most liked to least liked:\n"
        for _, row in remaining_5_summaries.iterrows():
            prompt += f"Movie: {row['Movie_ID']}, Summary: {row['Llama Summary Few Shot']}\n"

        prompt += (
            "\nReturn ONLY a Python list of Movie IDs in ranked order like this:\n"
            "[173, 257, 110, 13, 580]\n"
            "DO NOT include any other text or explanation."
        )

        # Get response
        ai_msg = llm.invoke([
            ("system", "You are a movie recommendation engine. Rank only. Return only a list."),
            ("human", prompt)
        ])

        output = ai_msg.content.strip()
        if output.startswith("[") and output.endswith("]"):
            predicted = [int(x.strip()) for x in output[1:-1].split(",")]

            true_sorted = remaining_5[['item id', 'rating']].sort_values(by='rating', ascending=False)
            ground_truth = true_sorted['item id'].tolist()

            if sorted(predicted) != sorted(ground_truth):
                print(f"⚠️ Movie mismatch for user {user_id}, skipping...")
                continue

            predicted_rankings[user_id] = predicted
            buffer.append([user_id, predicted])
            print(f"🟢 User {user_id} predicted")

            # Write every 10 users
            if len(buffer) >= 10:
                df_to_save = pd.DataFrame(buffer, columns=["user_id", "predicted_ranking"])
                if not os.path.exists(csv_file):
                    df_to_save.to_csv(csv_file, index=False)
                else:
                    df_to_save.to_csv(csv_file, mode='a', header=False, index=False)
                buffer.clear()

                # Print changes saved
                total_predicted = len(predicted_rankings)
                print(f"✅ Changes saved! Total movies predicted: {total_predicted}")

        else:
            print(f"❌ Invalid format for user {user_id}: {output}")

    except Exception as e:
        print(f"❌ Error for user {user_id}: {e}")

# Save remaining predictions at the end
if buffer:
    df_to_save = pd.DataFrame(buffer, columns=["user_id", "predicted_ranking"])
    if not os.path.exists(csv_file):
        df_to_save.to_csv(csv_file, index=False)
    else:
        df_to_save.to_csv(csv_file, mode='a', header=False, index=False)

    # Print final changes saved
    total_predicted = len(predicted_rankings)
    print(f"✅ Changes saved! Total movies predicted: {total_predicted}")

print("\n✅ All users finished!")


⚠️ Movie mismatch for user 246, skipping...
⚠️ Movie mismatch for user 59, skipping...
⚠️ Movie mismatch for user 279, skipping...
⚠️ Movie mismatch for user 41, skipping...
⚠️ Movie mismatch for user 256, skipping...
⚠️ Movie mismatch for user 61, skipping...
⚠️ Movie mismatch for user 283, skipping...
⚠️ Movie mismatch for user 351, skipping...
⚠️ Movie mismatch for user 382, skipping...
⚠️ Movie mismatch for user 407, skipping...
⚠️ Movie mismatch for user 425, skipping...
⚠️ Movie mismatch for user 455, skipping...
🟢 User 563 predicted
🟢 User 558 predicted
🟢 User 564 predicted
🟢 User 573 predicted
🟢 User 549 predicted
🟢 User 567 predicted
🟢 User 569 predicted
🟢 User 562 predicted
🟢 User 576 predicted
🟢 User 577 predicted
✅ Changes saved! Total movies predicted: 372
⚠️ Movie mismatch for user 579, skipping...
🟢 User 555 predicted
🟢 User 572 predicted
🟢 User 575 predicted
🟢 User 584 predicted
🟢 User 588 predicted
🟢 User 568 predicted
🟢 User 586 predicted
🟢 User 591 predicted
🟢 User 5

In [ ]:
len(df['user id'].unique())

943

In [ ]:
len(predicted_rankings)

2

In [ ]:
# Load your data
df = pd.read_csv("u.csv")
predicted_df = pd.read_csv("predicted_rankings.csv")

# Create dictionary for ground truths
ground_truth_rankings = {}

for user_id in predicted_df["user_id"]:
    user_movies = df[df['user id'] == user_id]
    if len(user_movies) < 20:
        continue

    sorted_movies = user_movies.sort_values(by='rating', ascending=False)
    remaining_5 = sorted_movies.tail(5)

    true_sorted = remaining_5[['item id', 'rating']].sort_values(by='rating', ascending=False)
    ground_truth = true_sorted['item id'].tolist()
    ground_truth_rankings[user_id] = ground_truth

print(f"✅ Ground truth reconstructed for {len(ground_truth_rankings)} users")


✅ Ground truth reconstructed for 616 users


In [ ]:
ground_truth_rankings

{196: [580, 257, 13, 173, 110],
 186: [288, 258, 1083, 12, 269],
 22: [932, 688, 926, 405, 998],
 244: [732, 468, 235, 144, 121],
 166: [294, 748, 346, 286, 687],
 298: [742, 679, 275, 91, 276],
 115: [237, 69, 596, 969, 496],
 62: [569, 1028, 271, 931, 924],
 286: [229, 401, 325, 790, 455],
 200: [982, 48, 235, 1028, 934],
 210: [230, 926, 662, 243, 763],
 224: [92, 980, 715, 544, 991],
 303: [398, 145, 63, 1086, 363],
 122: [214, 1268, 69, 57, 11],
 194: [582, 756, 1206, 67, 228],
 234: [12, 243, 403, 557, 874],
 119: [755, 916, 294, 410, 931],
 299: [235, 94, 17, 1227, 749],
 308: [472, 1028, 452, 97, 309],
 95: [768, 573, 715, 398, 552],
 160: [230, 864, 969, 21, 832],
 301: [1230, 423, 395, 411, 53],
 225: [1443, 286, 479, 245, 143],
 290: [650, 151, 473, 449, 180],
 181: [1137, 1358, 1317, 1352, 544],
 278: [258, 882, 245, 302, 301],
 276: [43, 1095, 796, 1413, 1090],
 7: [669, 440, 324, 70, 145],
 10: [319, 663, 462, 430, 198],
 201: [777, 116, 702, 590, 1355],
 287: [240, 895, 